#### Requirements:
###### pip install torch_geometric
###### pip install torch_cluster
###### probably also torch>= 2.0

In [12]:
#from source import Dataset
#from source import tools
import awkward as ak
import glob
import time

#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import optimizers
#from keras import layers

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch_geometric.data import Data

'''
For handling graphs in pytorch we usually resort to the pytorch geometric library.
The full documentation can be found here: https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

Standard way to represent a graph in general is a N x N matrix, where N is the number of nodes in the graph.
In pytorch geometric, we represent a graph as a tuple where the first element is a N x 2 matrix, where each column describes the linked nodes.
E.g. if you have a [1 0 1  matrix, it would translate to the following adj matrix: [[0 1 2 2]
                    0 1 1                                                           [0 1 0 1]]
                    0 0 0] 
'''

edge_index = torch.tensor([[0, 1, 1, 2],  # example in the pygeometric notation
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float) # your features

data = Data(x=x, edge_index=edge_index) # new data object containing the features and the edge index
print (data)

Data(x=[3, 1], edge_index=[2, 4])


In [28]:
'''
If you want to build the adj matrix starting from your features 
you can use some of the utilities provided by pygeometric
'''
x = torch.tensor([[-1.0, -1.0], [-1.0, 1.0], [1.0, -1.0], [1.0, 1.0], [2.0, 2.0], [-2.0, 2.0]]) # your features

'''this next lineassignes each graph to a batch. 
This is done because in pygeometric batches are not parallelized, but they are concatenated sequentially.
Of course this is slower, but as graph are tipically not padded and of irregular shape,
this is the only way to do it in torch.
So in the big tensor that will be created by pygeometric, 
this allows the model to distinguish which graphs belogs to which batch.
'''
batch = torch.tensor([0, 0, 0, 0, 1, 1])    

from torch_geometric.nn import knn_graph
#now you can build your adj matrix using the knn_graph 
#function https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.pool.knn_graph.html?highlight=knn_graph#torch_geometric.nn.pool.knn_graph
edge_index = knn_graph(x, k=2, batch=batch, loop=False)
#one can also use other functions to build the adj matrix, more info in the documentation


#finally you can create your data object like before
data = Data(x=x, edge_index=edge_index) # new data object containing the features and the edge index
print (data)


Data(x=[6, 2], edge_index=[2, 10])


In [29]:
print(edge_index)

tensor([[1, 2, 0, 3, 0, 3, 1, 2, 5, 4],
        [0, 0, 1, 1, 2, 2, 3, 3, 4, 5]])


In [30]:
path = "/nfs/dust/cms/user/lbenato/ML_LLP/MDS_regression/datasets/test_rechits_rearranged.h5"
store = pd.HDFStore(path)
df = store.select("df",start=0,stop=-1)

In [39]:
R = 3
max_h = -1
#take only 3 hits for simplicity; otherwise remove [0:3]
#phi = ak.Array([ df["cscClusteredRechitsPhi"].loc[nz:nz+1][0:3].values for nz in range(R)  ])

eta = ak.Array([ df["cscClusteredRechitsEta"].loc[nz:nz+1][0:max_h].values for nz in range(R)  ])
phi = ak.Array([ df["cscClusteredRechitsPhi"].loc[nz:nz+1][0:max_h].values for nz in range(R)  ])

points = ak.Array([ df[["cscClusteredRechitsEta","cscClusteredRechitsPhi"]].loc[nz:nz+1][0:max_h].values for nz in range(R)  ])
combi = ak.combinations(points,2, fields=["p0", "p1"])
combi_arg = ak.argcombinations(points,2)
combi_eta = ak.combinations(eta,2,fields=["p0", "p1"])
combi_phi = ak.combinations(phi,2,fields=["p0", "p1"])
print(len(combi))

print(points)
print(combi)
print(combi["p0"])
print(combi["p1"])


print("Now I need to build all the possible combinations among points, one per event")

3
[[[-1.07, -2.74], [-1.07, -2.77], ..., [-1.56, ...], [-1.53, -2.72]], ...]
[[{p0: [-1.07, -2.74], p1: [-1.07, ...]}, ..., {p0: [...], p1: ..., ...}], ...]
[[[-1.07, -2.74], [-1.07, -2.74], ..., [-1.57, ...], [-1.56, -2.71]], ...]
[[[-1.07, -2.77], [-1.07, -2.76], ..., [-1.53, ...], [-1.53, -2.72]], ...]
Now I need to build all the possible combinations among points, one per event


In [40]:
def deltaR_2(eta1, phi1, eta2, phi2):
    #print(eta1)
    #print(eta2)
    deta = eta1 - eta2
    #print(deta)
    dphi = np.arctan2(np.sin(phi1 - phi2), np.cos(phi1 - phi2))
    #print(dphi)
    return np.sqrt(deta**2 + dphi**2)

In [41]:
dR2 = deltaR_2(combi_eta["p0"],combi_phi["p0"],combi_eta["p1"],combi_phi["p1"])
#print(ak.Array(dR2))

distance = dR2
distance

<Array [[0.033, 0.0227, ..., 0.037, 0.0375], ...] type='3 * var * float64'>

In [42]:
print("------------------")
K=20
print("Sorting the combination indices based on their minimum delta R, and retain only the first ", K)
indices =  ak.sort(ak.argsort(distance,axis=1)[:,0:K])
print(indices)
print("Print eta/phi of the selected neighbour combinations")
#print(combi)
print(combi[indices])
print("Node indices of the selected neighbour combinations")
print("This does not include self-interactions")
#print(combi_arg)
nn_indices = np.array(combi_arg[indices])
print(nn_indices)
#print(nn_indices.shape)
#print(np.array(nn_indices.tolist()))
#print("swapaxes")
final_indices = np.swapaxes(np.array(nn_indices.tolist()),1,2).reshape(R,2,K)
print( final_indices[:,:,0:5] )

------------------
Sorting the combination indices based on their minimum delta R, and retain only the first  20
[[61617, 98397, 104688, 108072, ..., 126077, 127503, 127650, 128768], ...]
Print eta/phi of the selected neighbour combinations
[[{p0: [-1.6, -2.8], p1: [-1.6, -2.8]}, ..., {p0: [...], p1: [...]}], ...]
Node indices of the selected neighbour combinations
This does not include self-interactions
[[( 141,  142) ( 261,  262) ( 288,  289) ( 304,  305) ( 322,  323)
  ( 335,  336) ( 347,  348) ( 351,  352) ( 399,  400) ( 399,  401)
  ( 400,  401) ( 401,  402) ( 402,  403) ( 411,  412) ( 412,  414)
  ( 413,  414) ( 434,  435) ( 457,  458) ( 460,  461) ( 503,  504)]
 [(  16,   17) (  31,   32) (  31,   33) (  31,   34) (  32,   33)
  (  32,   34) (  33,   34) (  34,   35) (  43,   44) (  43,   46)
  (  44,   45) (  44,   46) (  45,   46) (  65,   67) (  66,   67)
  (  89,   90) (  92,   93) ( 135,  136) ( 145,  146) ( 223,  224)]
 [(   4,    5) ( 539,  540) ( 557,  558) ( 573,  574) 

In [88]:
from torch_geometric.data import Data

#create batch vector for knn
R = ak.num(points)
N = len(R)
i_batch = []
for n in range(N):
    i_batch += np.repeat(n,R[n]).tolist()
i_batch = np.array(i_batch)

x = torch.tensor(ak.flatten(points)) # your features
batch = torch.tensor(i_batch)    

from torch_geometric.nn import knn_graph
#now you can build your adj matrix using the knn_graph 
#function https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.pool.knn_graph.html?highlight=knn_graph#torch_geometric.nn.pool.knn_graph
edge_index = knn_graph(x, k=K, batch=batch, loop=False)
print(x)
print(edge_index)
print(batch)
#print(final_indices)
#one can also use other functions to build the adj matrix, more info in the documentation

#finally you can create your data object like before
data = Data(x=x, edge_index=edge_index) # new data object containing the features and the edge index
print(data)
print (data.edge_index)

#warning: this currently gives a different structure because it's not "batched"!
data = Data(x=x, edge_index=torch.tensor(final_indices))
print(data)
print(data.edge_index)

tensor([[-1.0680, -2.7377],
        [-1.0690, -2.7706],
        [-1.0688, -2.7604],
        ...,
        [ 2.3385, -2.3278],
        [ 2.3477, -2.3199],
        [ 2.3511, -2.3290]], dtype=torch.float64)
tensor([[   2,    3,    1,  ..., 2656, 2624, 2670],
        [   0,    0,    0,  ..., 2680, 2680, 2680]])
tensor([0, 0, 0,  ..., 2, 2, 2])
Data(x=[2681, 2], edge_index=[2, 53620])
tensor([[   2,    3,    1,  ..., 2656, 2624, 2670],
        [   0,    0,    0,  ..., 2680, 2680, 2680]])
Data(x=[2681, 2], edge_index=[3, 2, 20])
tensor([[[ 141,  261,  288,  304,  322,  335,  347,  351,  399,  399,  400,
           401,  402,  411,  412,  413,  434,  457,  460,  503],
         [ 142,  262,  289,  305,  323,  336,  348,  352,  400,  401,  401,
           402,  403,  412,  414,  414,  435,  458,  461,  504]],

        [[  16,   31,   31,   31,   32,   32,   33,   34,   43,   43,   44,
            44,   45,   65,   66,   89,   92,  135,  145,  223],
         [  17,   32,   33,   34,   33,   34,  

In [87]:
from torch_geometric.data import Data
x = torch.tensor(points[0]) # your features
#i_batch = np.repeat(np.arange(R),1)
#print(x.shape)
#print(i_batch)
#batch = torch.tensor(i_batch)    

from torch_geometric.nn import knn_graph
#now you can build your adj matrix using the knn_graph 
#function https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.pool.knn_graph.html?highlight=knn_graph#torch_geometric.nn.pool.knn_graph
edge_index = knn_graph(x, k=K, loop=False)
#print(x)
#print(edge_index)
#print(final_indices)
#one can also use other functions to build the adj matrix, more info in the documentation


#finally you can create your data object like before
data = Data(x=x, edge_index=edge_index) # new data object containing the features and the edge index
print(data)
print (data.edge_index)

data = Data(x=x, edge_index=torch.tensor(final_indices[0]))
print(data)
print(data.edge_index)

Data(x=[508, 2], edge_index=[2, 10160])
tensor([[  2,   3,   1,  ..., 326, 363, 324],
        [  0,   0,   0,  ..., 507, 507, 507]])
Data(x=[508, 2], edge_index=[2, 20])
tensor([[141, 261, 288, 304, 322, 335, 347, 351, 399, 399, 400, 401, 402, 411,
         412, 413, 434, 457, 460, 503],
        [142, 262, 289, 305, 323, 336, 348, 352, 400, 401, 401, 402, 403, 412,
         414, 414, 435, 458, 461, 504]])
